In [1]:
import numpy as np

import python_inventory as pyinv

In [3]:
class MultFunc:
    #
    def evaluate(self, X):
        pass
    #
    def gradient(self, X):
        pass

class Minimal(MultFunc):
    #
    def evaluate(self, X):
        return np.min(X)
    #
    def gradient(self, X):
        Is = np.argmin(X)
        v = 1.0 / len(Is)
        ret = np.zeros(len(X))
        for i in Is:
            ret[i] = v
        return ret
        
class SoftMinimal(MultFunc):
    #
    def __init__(self, a=1.0):
        self.a = a
    #
    def evaluate(self, X):
        return -np.log(np.sum(np.exp(-self.a * X)))
    #
    def gradient(self, X):
        V = np.exp(-self.a * X)
        S = np.sum(V)
        V *= self.a
        return V / S
        

class KMeansRegression:
    #
    def __init__(self, models, loss_func=None, agg=None, min_func=None, h=0.1, tol=1.0e-8, n_iter=100):
        self.models = models
        if loss_func is None:
            self.loss_func = pyinv.ErrorLoss(pyinv.Square())
        else:
            self.loss_func = loss_func
        if agg is None:
            self.agg_func = pyinv.ArithMean()
        else:
            self.agg_func = agg
        if minfunc is None:
            self.min_func = Minimal()
        else:
            self.min_func = min_func
        self.h = h
        self.tol = tol
        self.n_iter = n_iter
    #
    def fit(self, X, Y):
        models = self.models
        loss_func = self.loss_func
        min_func = self.min_func
        agg_func = self.agg_func

        m = len(models)
        N = len(X)

        algs = []
        for mod in models:
            rs = pyinv.Risk(mod, loss_func, agg_func)
            alg = pyinv.GradientDescent(rs, h=self.h, n_iter=self.n_iter, tol=self.tol)
            algs.append(alg)
        
        for K in range(self.n_iter):
            L = []
            GL = []
            for Xk, yk in zip(X, Y):
                loss_vals = np.from_iter((loss_func(mod.evaluate(Xk), yk) for mod in models), 'd', m)
                L.append(min_func.evaluate(loss_vals))
                GL.append(min_func.gradient(loss_vals))
            agg_func.evaluate(L)
            G = agg_func.gradient(L)
            
            for j, alg_j in enumerate(algs):
                Gj = []
                for g, gj in zip(G, GL):
                    G_j.append(g * g_j[j])
                alg_j.use_weights(Gj)
            
            
                